In [8]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/ML-Climate-Predicting-Wildfires/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

# tfds.disable_progress_bar()

import matplotlib.pyplot as plt
import numpy as np

In [14]:
import tensorflow as tf
INPUT_WIDTH = 128
INPUT_HEIGHT = 128
N_CHANNELS = 12
N_CLASSES = 2

In [6]:
def load_dir(data_dir):
  id_img_map = {}

  for f in os.listdir(data_dir):
    if '_' in f and not 'download_log' in f:
      id = f[:f.index('_')]
      if id in id_img_map:
        id_img_map[id].append(f)
      else:
        id_img_map[id] = [f]
  return id_img_map

data_dir = "data/TrainingData/Fire-Positive-5mo/"

In [2]:
def scale_resize_image(image):
    image = tf.image.convert_image_dtype(image, tf.float32) # equivalent to dividing image pixels by 255
    image = tf.image.resize(image, (INPUT_WIDTH, INPUT_HEIGHT)) # Resizing the image to 224x224 dimention
    return image

In [11]:
from tqdm import tqdm
import numpy as np

def load_dataset(id_img_map):
  images = []
  for id, fs in tqdm(list(id_img_map.items())):
    for i,f in enumerate(fs[::-1]): # Reverse reversed time
      image = np.load(data_dir + f)
      image_resized = scale_resize_image(image)
      images.append(image_resized)
  return np.array(images)

In [12]:
id_ig_map = load_dir(data_dir)

In [ ]:
X_positive = load_dataset(id_ig_map)

 86%|████████▌ | 2259/2633 [05:34<01:43,  3.60it/s]

In [18]:
X_positive.shape

array([], dtype=float64)

In [ ]:
with open('{}/Fire-Positive-Dataset.npy'.format(data_dir), 'wb') as f:
  np.save(f, positive_X)

In [ ]:
import os
import numpy as np

data_dir = 'data/TrainingData/Fire-Positive-New'
positive_X = np.load('{}/Fire-Positive-Dataset.npy'.format(data_dir))

In [ ]:
positive_Y = np.ones((positive_X.shape[0], 1))
positive_Y.shape

(4504, 1)

In [ ]:
data_dir = 'data/TrainingData/Fire-Negative'
print(len(os.listdir(data_dir)))
negative_data = []
for f in os.listdir(data_dir):
  if '.npy' in f:
    arr = np.load('{}/{}'.format(data_dir, f))
    image = scale_resize_image(arr)
    negative_data.append(image)

1319


In [ ]:
negative_X = np.array(negative_data)
negative_X.shape

(1318, 200, 200, 7)

In [ ]:
with open('{}/Fire-Negative-Dataset.npy'.format(data_dir), 'wb') as f:
  np.save(f, negative_X)

In [ ]:
data_dir = 'data/TrainingData/Fire-Negative'
negative_X = np.load('{}/Fire-Negative-Dataset.npy'.format(data_dir))

In [ ]:
negative_Y = np.zeros((negative_X.shape[0], 1))
negative_Y.shape

(1318, 1)

In [ ]:
X = np.concatenate((positive_X[:negative_X.shape[0]], negative_X), axis=0)
y = np.concatenate((positive_Y[:negative_Y.shape[0]], negative_Y), axis=0)
X.shape, y.shape

((2636, 200, 200, 7), (2636, 1))

In [ ]:
with open('data/TrainingData/DatasetX.npy', 'wb') as f:
  np.save(f, X)
with open('data/TrainingData/DatasetY.npy', 'wb') as f:
  np.save(f, y)

In [ ]:
import os
import numpy as np

X = np.load('data/TrainingData/DatasetX.npy')
y = np.load('data/TrainingData/DatasetY.npy')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

In [ ]:
train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
tf.data.experimental.save(
    train, 'data/TrainingData/train', compression=None, shard_func=None, checkpoint_args=None
)
tf.data.experimental.save(
    val, 'data/TrainingData/val', compression=None, shard_func=None, checkpoint_args=None
)
tf.data.experimental.save(
    test, 'data/TrainingData/test', compression=None, shard_func=None, checkpoint_args=None
)

In [ ]:
train = tf.data.experimental.load(
    'data/TrainingData/train', element_spec=None, compression=None, reader_func=None
)

val = tf.data.experimental.load(
    'data/TrainingData/val', element_spec=None, compression=None, reader_func=None
)

test = tf.data.experimental.load(
    'data/TrainingData/test', element_spec=None, compression=None, reader_func=None
)

In [ ]:
BATCH_SIZE = 5
SHUFFLE_BUFFER_SIZE = 10

train = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val = val.batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)

In [ ]:
from keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
def build_convnet(shape=(INPUT_WIDTH, INPUT_HEIGHT, N_CHANNELS)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # model.add(MaxPool2D())
    
    # model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    # model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    # model.add(BatchNormalization(momentum=momentum))
    
    # model.add(MaxPool2D())
    
    # model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    # model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    # model.add(BatchNormalization(momentum=momentum))
    
    # model.add(MaxPool2D())
    
    # model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    # model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    # model.add(BatchNormalization(momentum=momentum))
    
    # # flatten...
    # model.add(GlobalMaxPool2D())
    return model

/usr/local/lib/python3.7/dist-packages/keras/applications/mobilenet_v2.py:285: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 7 input channels.
  weights=weights)


In [ ]:
from keras.layers import LSTM, GRU, Dense, Dropout

def action_model(shape=(6, INPUT_WIDTH, INPUT_HEIGHT, N_CHANNELS), nbout=2):
    # Create our convnet with (128, 128, 12) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (6, 128, 128, 12) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(LSTM(64))
    model.add(Dropout(.3))
    # and finally, we make a decision network
    model.add(Dense(264, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

NBFRAME = 6
INSHAPE=(NBFRAME,) + (INPUT_WIDTH, INPUT_HEIGHT) + (N_CHANNELS,) # (6, 128, 128, 12)
model = action_model(INSHAPE, len(classes), nbout=1)
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'binary_crossentropy',
    metrics=['acc']
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 16)      1024      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 64)      

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
optimizer=RMSprop(lr=0.001),
metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
len(train)

317

In [ ]:
history = model.fit(train,
steps_per_epoch=10,
epochs=30,
verbose=1,
validation_data = val,
validation_steps=8)

Epoch 1/30
10/10 [==============================] - 8s 805ms/step - loss: 0.7371 - accuracy: 0.6600 - precision: 0.6957 - recall: 0.6154 - val_loss: 0.6282 - val_accuracy: 0.6500 - val_precision: 0.8889 - val_recall: 0.5714
Epoch 2/30
10/10 [==============================] - 5s 551ms/step - loss: 0.6208 - accuracy: 0.7200 - precision: 0.7727 - recall: 0.6538 - val_loss: 0.5084 - val_accuracy: 0.7500 - val_precision: 0.7812 - val_recall: 0.8929
Epoch 3/30
10/10 [==============================] - 6s 585ms/step - loss: 0.6711 - accuracy: 0.6600 - precision: 0.6667 - recall: 0.8667 - val_loss: 0.5450 - val_accuracy: 0.7250 - val_precision: 0.8696 - val_recall: 0.7143
Epoch 4/30
10/10 [==============================] - 5s 558ms/step - loss: 0.7017 - accuracy: 0.6400 - precision: 0.5333 - recall: 0.4211 - val_loss: 0.5789 - val_accuracy: 0.7750 - val_precision: 0.9130 - val_recall: 0.7500
Epoch 5/30
10/10 [==============================] - 6s 566ms/step - loss: 0.6545 - accuracy: 0.6000 - pr

In [ ]:
loss, accuracy, precision, recall = model.evaluate(test, verbose=0)

In [ ]:
loss, accuracy, precision, recall

(0.5874483585357666,
 0.6931818127632141,
 0.8137931227684021,
 0.46640315651893616)

[Medium article](https://medium.com/smileinnovation/training-neural-network-with-image-sequence-an-example-with-video-as-input-c3407f7a0b0f)

In [ ]:
# from keras.layers import TimeDistributed, GRU, Dense, Dropout
# def action_model(shape=(5, 112, 112, 3), nbout=3):
#     # Create our convnet with (112, 112, 3) input shape
#     convnet = build_convnet(shape[1:])
    
#     # then create our final model
#     model = keras.Sequential()
#     # add the convnet with (5, 112, 112, 3) shape
#     model.add(TimeDistributed(convnet, input_shape=shape))
#     # here, you can also use GRU or LSTM
#     model.add(GRU(64))
#     # and finally, we make a decision network
#     model.add(Dense(1024, activation='relu'))
#     model.add(Dropout(.5))
#     model.add(Dense(512, activation='relu'))
#     model.add(Dropout(.5))
#     model.add(Dense(128, activation='relu'))
#     model.add(Dropout(.5))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(nbout, activation='softmax'))
#     return model


In [ ]:
# INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
# model = action_model(INSHAPE, len(classes))
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer,
    'binary_crossentropy',
    metrics=['acc']
)

In [ ]:
EPOCHS=50
# create a "chkp" directory before to run that
# because ModelCheckpoint will write models inside
callbacks = [
    keras.callbacks.ReduceLROnPlateau(verbose=1),
    keras.callbacks.ModelCheckpoint(
        'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
        verbose=1),
]
model.fit_generator(
    train,
    validation_data=val,
    verbose=1,
    epochs=EPOCHS,
    callbacks=callbacks
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  from ipykernel import kernelapp as app


Epoch 1/50
210/210 [==============================] - ETA: 0s - loss: 6.5130 - acc: 0.4279
Epoch 1: saving model to chkp/weights.01-5.81.hdf5
210/210 [==============================] - 234s 1s/step - loss: 6.5130 - acc: 0.4279 - val_loss: 5.8113 - val_acc: 0.3811 - lr: 0.0010
Epoch 2/50
210/210 [==============================] - ETA: 0s - loss: 6.5104 - acc: 0.4269
Epoch 2: saving model to chkp/weights.02-5.81.hdf5
210/210 [==============================] - 235s 1s/step - loss: 6.5104 - acc: 0.4269 - val_loss: 5.8113 - val_acc: 0.3811 - lr: 0.0010
Epoch 3/50
210/210 [==============================] - ETA: 0s - loss: 6.5104 - acc: 0.4269
Epoch 3: saving model to chkp/weights.03-5.81.hdf5
210/210 [==============================] - 231s 1s/step - loss: 6.5104 - acc: 0.4269 - val_loss: 5.8113 - val_acc: 0.3811 - lr: 0.0010
Epoch 4/50
210/210 [==============================] - ETA: 0s - loss: 6.5104 - acc: 0.4269
Epoch 4: saving model to chkp/weights.04-5.81.hdf5
210/210 [==================

KeyboardInterrupt: ignored

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
def build_mobilenet(shape=(224, 224, 3), nbout=3):
    model = keras.applications.mobilenet.MobileNet(
        include_top=False,
        input_shape=shape,
        weights='imagenet')
    # Keep 9 layers to train﻿﻿
    trainable = 9
    for layer in model.layers[:-trainable]:
        layer.trainable = False
    for layer in model.layers[-trainable:]:
        layer.trainable = True
    output = keras.GlobalMaxPool2D()
    return keras.Sequential([model, output])

In [ ]:
# Set size to 224, 224
SIZE = (224, 224)
CHANNELS = 3
NBFRAME = 5
BS = 8

In [ ]:
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_mobilenet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [ ]:
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 224, 224, 3)
model = action_model(INSHAPE, len(classes))
optimizer = keras.optimizers.SGD()
model.compile(
    optimizer,
    'categorical_crossentropy',
    metrics=['acc']
)